In [10]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook


In [11]:
with open('/Users/Steven (Dad)/.secret/yelp_api.json') as (f):
    login =json.load(f)
    
yelp_api = YelpAPI(login['api-key'], timeout_s = 5.0)

In [12]:
LOCATION = 'Phoenix, AZ'
TERM = 'Sushi'



In [13]:
JSON_FILE = f"Data/results_in_progress_Sushi.json"
JSON_FILE

'Data/results_in_progress_Sushi.json'

In [14]:
file_exists = os.path.isfile(JSON_FILE)

if file_exists == False:
    folder = os.path.dirname(JSON_FILE)
    
    if len(folder)>0:
        os.makedirs(folder,exist_ok=True)
        
    print(f"[i] {JSON_FILE} not found. Saving empty list to file.")
    
    ## save the first page of results
    with open(JSON_FILE,'w') as f:
        json.dump([],f)  
## If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_in_progress_Sushi.json not found. Saving empty list to file.


In [15]:
with open(JSON_FILE, 'r') as f:
    previous_results = json.load(f)
    
n_results = len(previous_results)
print(f'- {n_results} previous results found.')


- 0 previous results found.


In [16]:
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [17]:
total_results = results['total']
total_results

836

In [18]:
results_per_page = len(results['businesses'])
results_per_page

20

In [19]:
n_pages = math.ceil((results['total']-n_results)/ results_per_page)
n_pages

42

In [20]:
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f)

In [21]:
for i in tqdm_notebook(range(n_pages)):
    # adds 200 ms pause
    time.sleep(.2) 

  0%|          | 0/42 [00:00<?, ?it/s]

In [22]:
for i in tqdm_notebook( range(1,n_pages+1)):
    time.sleep(.2)
    ## Read in results in progress file and check the length
    with open(JSON_FILE, 'r') as f:
        previous_results = json.load(f)
    ## save number of results for to use as offset
    n_results = len(previous_results)
    ## use n_results as the OFFSET 
    results = yelp_api.search_query(location=LOCATION,
                                    term=TERM, 
                                    offset=n_results)
    
    ## append new results and save to file
    previous_results.extend(results['businesses'])
    
#     display(previous_results)
    with open(JSON_FILE,'w') as f:
        json.dump(previous_results,f)

  0%|          | 0/42 [00:00<?, ?it/s]

In [23]:
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,_n0B8mRTWP6ZfX8npDGEqw,yutaka-japanese-restaurant-phoenix,Yutaka Japanese Restaurant,https://s3-media1.fl.yelpcdn.com/bphoto/GFyq1d...,False,https://www.yelp.com/biz/yutaka-japanese-resta...,665,"[{'alias': 'ramen', 'title': 'Ramen'}, {'alias...",4.5,"{'latitude': 33.639089, 'longitude': -112.063179}","[delivery, pickup]",$$,"{'address1': '751 E Bell Rd', 'address2': 'Ste...",+16028438882,(602) 843-8882,2853.176900
1,S-oLPRdhlyL5HAknBKTUcQ,harumi-sushi-and-sake-downtown-phoenix,Harumi Sushi & Sake- Downtown,https://s3-media1.fl.yelpcdn.com/bphoto/RGzWKj...,False,https://www.yelp.com/biz/harumi-sushi-and-sake...,2116,"[{'alias': 'japanese', 'title': 'Japanese'}, {...",4.5,"{'latitude': 33.4497491809777, 'longitude': -1...","[delivery, pickup]",$$,"{'address1': '114 W Adams St', 'address2': 'St...",+16022580131,(602) 258-0131,20614.426085
2,evdJO0v9rvVixieNEnaeJg,sushiholic-phoenix,Sushiholic,https://s3-media2.fl.yelpcdn.com/bphoto/Fe1HNI...,False,https://www.yelp.com/biz/sushiholic-phoenix?ad...,1049,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",4.5,"{'latitude': 33.5096751, 'longitude': -111.995...",[delivery],$$,"{'address1': '3957 E Camelback Rd', 'address2'...",+16029562354,(602) 956-2354,16632.911261
3,XFO9us6shjSRwiuLBO9T9g,kura-revolving-sushi-bar-phoenix,Kura Revolving Sushi Bar,https://s3-media2.fl.yelpcdn.com/bphoto/ZnJz7M...,False,https://www.yelp.com/biz/kura-revolving-sushi-...,169,"[{'alias': 'sushi', 'title': 'Sushi Bars'}, {'...",3.5,"{'latitude': 33.507777974615145, 'longitude': ...",[],NaN,"{'address1': '1949 E Camelback Rd', 'address2'...",+15204792888,(520) 479-2888,14942.956277
4,itJJHcstARVqibAUtxjyUQ,o2-sushi-phoenix-2,O2 Sushi,https://s3-media2.fl.yelpcdn.com/bphoto/bWpCwm...,False,https://www.yelp.com/biz/o2-sushi-phoenix-2?ad...,217,"[{'alias': 'conveyorsushi', 'title': 'Conveyor...",4.5,"{'latitude': 33.6790557, 'longitude': -112.030...","[delivery, pickup]",$$,"{'address1': '21043 N Cave Creek Rd', 'address...",+16029712190,(602) 971-2190,7680.369098


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
831,N-GN8S5IUhBnQXK-sk7hGg,frys-marketplace-chandler-2,Fry's Marketplace,https://s3-media1.fl.yelpcdn.com/bphoto/qjlnAB...,False,https://www.yelp.com/biz/frys-marketplace-chan...,47,"[{'alias': 'drugstores', 'title': 'Drugstores'...",3.0,"{'latitude': 33.2175788879395, 'longitude': -1...",[],$$,"{'address1': '985 E Riggs Rd', 'address2': '',...",+14808836500,(480) 883-6500,52638.119166
832,NtvCX9UEWFvHSxQj06XQ0g,safeway-scottsdale-7,Safeway,https://s3-media1.fl.yelpcdn.com/bphoto/erXfhj...,False,https://www.yelp.com/biz/safeway-scottsdale-7?...,94,"[{'alias': 'grocery', 'title': 'Grocery'}]",3.0,"{'latitude': 33.619372453704706, 'longitude': ...",[delivery],$$,{'address1': '14696 N Frank Lloyd Wright Blvd'...,+14803911090,(480) 391-1090,20286.830657
833,ZSMvp7DrOydyr0IlZZLjKA,sprouts-farmers-market-mesa-3,SPROUTS FARMERS MARKET,https://s3-media4.fl.yelpcdn.com/bphoto/C1gjNf...,False,https://www.yelp.com/biz/sprouts-farmers-marke...,49,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",3.0,"{'latitude': 33.380802, 'longitude': -111.786934}",[],$$,"{'address1': '2060 E Baseline Rd', 'address2':...",+14804013758,(480) 401-3758,40047.965626
834,e755zXCj5hGy6-P-1otnKg,frys-marketplace-tempe,Fry's Marketplace,https://s3-media1.fl.yelpcdn.com/bphoto/GmT3n-...,False,https://www.yelp.com/biz/frys-marketplace-temp...,92,"[{'alias': 'grocery', 'title': 'Grocery'}, {'a...",3.0,"{'latitude': 33.3217415032877, 'longitude': -1...",[],$$,"{'address1': '9900 S Rural Rd', 'address2': ''...",+14807836200,(480) 783-6200,37940.479374
835,qdyYXsF7UzmU6s8vszs0tQ,frys-marketplace-phoenix-13,Fry's Marketplace,https://s3-media1.fl.yelpcdn.com/bphoto/f083L-...,False,https://www.yelp.com/biz/frys-marketplace-phoe...,67,"[{'alias': 'drugstores', 'title': 'Drugstores'...",3.5,"{'latitude': 33.30389, 'longitude': -111.99888}",[],$$,"{'address1': '3949 E Chandler Blvd', 'address2...",+14807067373,(480) 706-7373,37846.269792


In [24]:
final_df.duplicated(subset='id').sum()

0

In [25]:
final_df.to_csv('Data/final_results_Sushi.csv.gz', compression='gzip',index=False)